<img style="float: left;;" src='Figures/alinco.png' /></a>

# Actividad 5: WordEmbeddings (Modelo CBOW)



En esta acividad, practicarás cómo calcular wordembedding y utilizarlas para el análisis de sentimientos.

- Para implementar el análisis de sentimientos, podemos ir más allá de contar el número de palabras positivas y negativas.
- Podrás encontrar una manera de representar cada palabra numéricamente, mediante un vector.
- El vector podría entonces representar estructuras sintácticas (es decir, partes del discurso) y semánticas (es decir, significado).

En esta actividad, explorarás una forma clásica de generar wordembeddings de palabras.
- Implementarás un modelo famoso llamado modelo de bolsa continua de palabras (CBOW).


Saber cómo entrenar estos modelos te brindará una mejor comprensión de los vectores palabras, que son los componentes básicos de muchas aplicaciones en el procesamiento del lenguaje natural.

<a name='1'></a>
# Continuous bag of words (CBOW)

Observemos la siguiente sentencia:
>**'I am happy because I am learning'**.

- En el modelado de bolsa continua de palabras (CBOW), intentamos predecir la palabra central dadas algunas palabras de contexto (las palabras alrededor de la palabra central).

- Por ejemplo, si tuviera que elegir un contexto de tamaño medio, digamos $C = 2$, entonces intentaría predecir la palabra **happy** dado el contexto que incluye 2 palabras antes y 2 palabras después de la palabra central. :

> $C$ words before: [I, am]

> $C$ words after: [because, I]

- en otras palabras:

$$context = [I,am, because, I]$$
$$target = happy$$

La estructura del modelo se ve como sigue:

<div style="width:image width px; font-size:100%; text-align:center;"><img src='Figures/word2.png' alt="alternate text" width="width" height="height" style="width:600px;height:250px;" /> Figure 1 </div>

dond $\bar x$ es el promedio de todos los vectores one-hot de las palabras de contexto.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='mean_vec2.png' alt="alternate text" width="width" height="height" style="width:600px;height:250px;" /> Figure 2 </div>

Una vez que ya tenemos los vectores contextos, podemos usar $\bar x$  como la entrada al modelo

La arquitectura a implementar es la siguiente:

\begin{align}
 h &= W_1 \  X + b_1  \tag{1} \\
 a &= ReLU(h)  \tag{2} \\
 z &= W_2 \  a + b_2   \tag{3} \\
 \hat y &= softmax(z)   \tag{4} \\
\end{align}

In [1]:
# Importar librerias y funciones de ayuda (utils2)
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
from collections import Counter
from utils2 import sigmoid, get_batches, compute_pca, get_dict

In [2]:

nltk.data.path.append('.')

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# cargar y tokenizar
import re

In [5]:
# Leer el archivo txt shakespeare.txt
with open('shakespeare.txt') as f:
  data = f.read()
#Limpiar los elementos del archivo
data = re.sub(r'[,!?;-]', '.', data)      #remplazando símbolos de puntuación por un .
data = nltk.word_tokenize(data)           #Tokenizando las palabras
data = [ch.lower() for ch in data if ch.isalpha() or ch=='.']
print('Num de tokens: ', len(data),'\n', data[:15])

Num de tokens:  60976 
 ['o', 'for', 'a', 'muse', 'of', 'fire', '.', 'that', 'would', 'ascend', 'the', 'brightest', 'heaven', 'of', 'invention']


In [7]:
# obtener la distribucion de frecuencias en el vocabulario
fdist = nltk.FreqDist(word for word in data)
print('tamaño del vocabulario (V): ', len(fdist))
print('algunas de los tokens más frecuentes son: ', fdist.most_common(20))

tamaño del vocabulario (V):  5775
algunas de los tokens más frecuentes son:  [('.', 9630), ('the', 1521), ('and', 1394), ('i', 1257), ('to', 1159), ('of', 1093), ('my', 857), ('that', 781), ('in', 770), ('a', 752), ('you', 748), ('is', 630), ('not', 559), ('for', 467), ('it', 460), ('with', 441), ('his', 434), ('but', 417), ('me', 417), ('your', 397)]


#### Mapear las palabras a sus indices y de indices a palabras


In [8]:
# get_dict obtener los diccionarios ind2vec, vec2ind
word2Ind, Ind2word = get_dict(data)

In [9]:
word2Ind['king']

2744

In [11]:
Ind2word[2744]

'king'

In [12]:
V = len(word2Ind)
V

5775

<a name='2'></a>
# Entrenando el Modelo

###  Inicialización del modelo

Ahora inicializarás dos matrices y dos vectores.
- La primera matriz ($W_1$) es de dimensión $N \times V$, donde $V$ es el número de palabras en tu vocabulario y $N$ es la dimensión de tu vector de palabras.
- La segunda matriz ($W_2$) es de dimensión $V \times N$.
- El vector $b_1$ tiene dimensiones $N\times 1$
- El vector $b_2$ tiene dimensiones $V\times 1$.
- $b_1$ y $b_2$ son los vectores bias.

La estructura general del modelo se verá como en la Figura 1, pero en esta etapa solo estamos inicializando los parámetros.


In [13]:
def initialize_model(N,V, random_seed=1):
    '''
    Inputs:
        N:  dimension of hidden vector
        V:  dimension of vocabulary
        random_seed: random seed for consistent results in the unit tests
     Outputs:
        W1, W2, b1, b2: initialized weights and biases
    '''

    np.random.seed(random_seed)

    # W1 shape (N,V)
    W1 = np.random.rand(N,V)
    # W2  shape (V,N)
    W2 = np.random.rand(V,N)
    # b1  shape (N,1)
    b1 = np.random.rand(N,1)
    # b2  shape (V,1)
    b2 = np.random.rand(V,1)

    return W1, W2, b1, b2

In [14]:
# Testear la función.
temp_N = 4
temp_V = 10
W1_temp, W2_temp, b1_temp, b2_temp = initialize_model(temp_N,temp_V, random_seed=1)

In [16]:
W1_temp.shape, W2_temp.shape, b1_temp.shape, b2_temp.shape

((4, 10), (10, 4), (4, 1), (10, 1))

In [17]:
W1_temp

array([[4.17022005e-01, 7.20324493e-01, 1.14374817e-04, 3.02332573e-01,
        1.46755891e-01, 9.23385948e-02, 1.86260211e-01, 3.45560727e-01,
        3.96767474e-01, 5.38816734e-01],
       [4.19194514e-01, 6.85219500e-01, 2.04452250e-01, 8.78117436e-01,
        2.73875932e-02, 6.70467510e-01, 4.17304802e-01, 5.58689828e-01,
        1.40386939e-01, 1.98101489e-01],
       [8.00744569e-01, 9.68261576e-01, 3.13424178e-01, 6.92322616e-01,
        8.76389152e-01, 8.94606664e-01, 8.50442114e-02, 3.90547832e-02,
        1.69830420e-01, 8.78142503e-01],
       [9.83468338e-02, 4.21107625e-01, 9.57889530e-01, 5.33165285e-01,
        6.91877114e-01, 3.15515631e-01, 6.86500928e-01, 8.34625672e-01,
        1.82882773e-02, 7.50144315e-01]])

<a name='2.1'></a>
### 2.1 Softmax
Antes de que podamos comenzar a entrenar el modelo, debemos implementar la función softmax como se define en la ecuación 5:  

<br>
$$ \text{softmax}(z_i) = \frac{e^{z_i} }{\sum_{i=0}^{V-1} e^{z_i} }  \tag{5} $$

- La indexación de matrices en el código comienza en 0.
- $V$ es el número de palabras en el vocabulario (que también es el número de filas de $z$).
- $i$ va de 0 a |V| - 1.


**Implemente la función softmax a continuación. **

- Supongamos que la entrada $z$ a `softmax` es una matriz 2D
- Cada ejemplo de entrenamiento está representado por una columna de forma (V, 1) en esta matriz 2D.
- Puede haber más de una columna en la matriz 2D, porque puede incluir un lote de ejemplos para aumentar la eficiencia. Llamemos al tamaño del lote $m$ en minúsculas, por lo que la matriz $z$ tiene la forma (V, m)
- Al tomar la suma de $i=1 \cdots V-1$, toma la suma de cada columna (cada ejemplo) por separado.


In [18]:
def softmax(z):
    '''
    Inputs:
        z: output scores from the hidden layer
    Outputs:
        yhat: prediction (estimate of y)
    '''
    e_z = np.exp(z)
    yhat = e_z/np.sum(e_z, axis=0)

    return yhat

In [19]:
# Testear la función
tmp = np.array([[1,2,3],
                [1,1,1]])
tmp_sm = softmax(tmp)

In [20]:
tmp_sm

array([[0.5       , 0.73105858, 0.88079708],
       [0.5       , 0.26894142, 0.11920292]])

<a name='2.2'></a>
### Forward propagation

Implemente la propagación directa $z$ según las ecuaciones (1) a (3). <br>

\begin{align}
 h &= W_1 \  X + b_1  \tag{1} \\
 a &= ReLU(h)  \tag{2} \\
 z &= W_2 \  a + b_2   \tag{3} \\
\end{align}

Para ello utilizarás como función de activación la ReLU dada por::

$$f(h)=\max (0,h) \tag{6}$$

In [21]:
def forward_prop(x, W1, W2, b1, b2):
    '''
    Inputs:
        x:  average one hot vector for the context
        W1, W2, b1, b2:  matrices and biases to be learned
     Outputs:
        z:  output score vector
    '''
    # Calcular h
    h = np.dot(W1,x) + b1

    #Aplicar la relu a h
    h = np.maximum(0,h)

    #Calcular z
    z = np.dot(W2, h) + b2

    return z, h

In [26]:
# Testear la función
tmp_N = 2
tmp_V = 3
tmp_x = np.array([[0,1,0]]).T
tmp_W1,tmp_W2,tmp_b1,tmp_b2 = initialize_model(N = tmp_N, V = tmp_V, random_seed=1)
#Llamar a la funcion forward_propagation
tmp_z, tmp_h = forward_prop(tmp_x, tmp_W1, tmp_W2, tmp_b1, tmp_b2)

In [27]:
tmp_z.shape, tmp_h.shape

((3, 1), (2, 1))

In [28]:
tmp_z

array([[0.55379268],
       [1.58960774],
       [1.50722933]])

In [29]:
tmp_h

array([[0.92477674],
       [1.02487333]])


## Función de Costo


In [30]:
# cross-entropy cost functioN
def compute_cost(y, yhat, batch_size):
    # cost function
    logprobs = np.multiply(np.log(yhat),y) + np.multiply(np.log(1 - yhat), 1 - y)
    cost = - 1/batch_size * np.sum(logprobs)
    cost = np.squeeze(cost)
    return cost

In [31]:
# Testear la función
tmp_C = 2 #ventana para las palanbras de contexto
tmp_N = 50
tmp_batch_size=4
tmp_word2ind, tmp_Ind2word = get_dict(data)
tmp_V = len(tmp_word2ind)
# Generacion de batches
tmp_x, tmp_y = next(get_batches(data, tmp_word2ind, tmp_V, tmp_C, tmp_batch_size))



In [32]:
tmp_x.shape, tmp_y.shape

((5775, 4), (5775, 4))

In [36]:
#Inicializar el modelo
tmp_W1,tmp_W2,tmp_b1,tmp_b2 = initialize_model(N = tmp_N, V = tmp_V, random_seed=1)

In [35]:
tmp_W1.shape, tmp_W2.shape, tmp_b1.shape, tmp_b2.shape

((50, 5775), (5775, 50), (50, 1), (5775, 1))

In [37]:
#Llama la funcion forward_prop
tmp_z, tmp_h = forward_prop(tmp_x, tmp_W1, tmp_W2, tmp_b1, tmp_b2)

In [38]:
tmp_z.shape, tmp_h.shape

((5775, 4), (50, 4))

In [39]:
#oBTENIENDO LA SALIDA con softmax
tmp_yhat = softmax(tmp_z)
tmp_yhat

array([[1.57362258e-04, 1.57362258e-04, 1.57362258e-04, 1.57362258e-04],
       [3.02069033e-05, 3.02069033e-05, 3.02069033e-05, 3.02069033e-05],
       [6.37370382e-05, 6.37370382e-05, 6.37370382e-05, 6.37370382e-05],
       ...,
       [1.61885109e-04, 1.61885109e-04, 1.61885109e-04, 1.61885109e-04],
       [1.86710849e-05, 1.86710849e-05, 1.86710849e-05, 1.86710849e-05],
       [6.17182995e-05, 6.17182995e-05, 6.17182995e-05, 6.17182995e-05]])

In [40]:
tmp_yhat.shape


(5775, 4)

In [41]:
#Obtener el valor de la función de costo
tmp_cost = compute_cost(tmp_y, tmp_yhat, tmp_batch_size)
tmp_cost

11.409481698203312


## Entrenar al modelo - Backpropagation

Ahora que has entendido cómo funciona el modelo CBOW, lo entrenarás. <br>
Creaste una función para la propagación hacia adelante. Ahora implementará una función que calcula los gradientes para propagar los errores hacia atrás.

In [42]:
def relu(z):
    result = z.copy()
    result[result<0]=0
    return result

In [43]:
l1=np.array([[0.52727857,  0.52727857,  0.52727857,  0.52727857],
 [-0.1259346,  -0.1259346,  -0.1259346 , -0.1259346 ],
 [ 0.39739328,  0.39739328,  0.39739328,  0.39739328],
 [-0.33644763, -0.33644763, -0.33644763, -0.33644763]])

In [44]:
relu(np.array([-1,0,0.2,-0.3]))

array([0. , 0. , 0.2, 0. ])

In [45]:
np.apply_along_axis(relu, 0, l1)

array([[0.52727857, 0.52727857, 0.52727857, 0.52727857],
       [0.        , 0.        , 0.        , 0.        ],
       [0.39739328, 0.39739328, 0.39739328, 0.39739328],
       [0.        , 0.        , 0.        , 0.        ]])

In [47]:
def back_prop(x, yhat, y, h, W1, W2, b1, b2, batch_size):
    '''
    Inputs:
        x:  average one hot vector for the context
        yhat: prediction (estimate of y)
        y:  target vector
        h:  hidden vector (see eq. 1)
        W1, W2, b1, b2:  matrices and biases
        batch_size: batch size
     Outputs:
        grad_W1, grad_W2, grad_b1, grad_b2:  gradients of matrices and biases
    '''
    l1 = np.dot(W2.T, (yhat-y))
    #aplicar la función de activación relu
    l1 = np.apply_along_axis(relu,0,l1)
    #Calcular el gradiente de w1
    grad_W1 = (1/batch_size)*np.dot(l1,x.T)
    #Calcular el gradiente de w2
    grad_W2 = (1/batch_size)*np.dot(yhat-y, h.T)
    #Calcular el gradiente de b1
    grad_b1 = np.sum((1/batch_size)*np.dot(l1,x.T), axis=1, keepdims=True)
    #Calcular el gradiente de b2
    grad_b2 = np.sum((1/batch_size)*np.dot(yhat-y,h.T), axis=1, keepdims=True)

    return grad_W1, grad_W2, grad_b1, grad_b2


In [48]:
# Testear la función
tmp_grad_W1, tmp_grad_W2, tmp_grad_b1, tmp_grad_b2 =back_prop(tmp_x, tmp_yhat, tmp_y, tmp_h,
                                                              tmp_W1, tmp_W2, tmp_b1, tmp_b2, tmp_batch_size)

In [49]:
tmp_grad_W1.shape, tmp_grad_W2.shape, tmp_grad_b1.shape, tmp_grad_b2.shape

((50, 5775), (5775, 50), (50, 1), (5775, 1))


## Gradient Descent

Ahora que ha implementado una función para calcular los gradientes, implementará el descenso de gradientes por **lotes** en su conjunto de entrenamiento.

**Hint:** Para eso, usarás `initialize_model` y las funciones `back_prop` que acabas de crear (y la función `compute_cost`). También puedes utilizar la función auxiliar `get_batches` proporcionada:

```for x, y in get_batches(data, word2Ind, V, C, batch_size):```

```...```
Además: imprima el costo después de procesar cada lote (use el tamaño de lote = 128)

In [52]:

def gradient_descent(data, word2Ind, N, V, num_iters, alpha=0.03):

    '''
    This is the gradient_descent function

      Inputs:
        data:      text
        word2Ind:  words to Indices
        N:         dimension of hidden vector
        V:         dimension of vocabulary
        num_iters: number of iterations
     Outputs:
        W1, W2, b1, b2:  updated matrices and biases

    '''
    W1,W2,b1,b2 = initialize_model(N,V, random_seed=282)
    batch_size=128
    iters=0
    C=2

    for x, y in get_batches(data, word2Ind, V, C, batch_size):
      z, h = forward_prop(x, W1,W2, b1, b2)
      yhat = softmax(z)
      cost = compute_cost(y, yhat, batch_size)

      if ((iters+1)%10 ==0):
        print(f'iteraciones: {iters+1} costo: {cost:.6f}')

      grad_W1, grad_W2, grad_b1, grad_b2 = back_prop(x, yhat, y, h, W1, W2, b1, b2, batch_size)

      #Actualizar los pesos
      W1-=alpha*grad_W1
      W2-=alpha*grad_W2
      b1-=alpha*grad_b1
      b2-=alpha*grad_b2

      iters+=1
      if iters == num_iters:
        break
      if iters %100==0:
        alpha *=0.66

    return W1, W2, b1, b2

In [53]:
# testear la función
C=2
N=50
word2Ind, Ind2word = get_dict(data)
V= len(word2Ind)
num_iters=150

W1, W2, b1, b2 = gradient_descent(data, word2Ind, N, V, num_iters, alpha=0.03)

iteraciones: 10 costo: 0.106780
iteraciones: 20 costo: 0.040070
iteraciones: 30 costo: 0.024766
iteraciones: 40 costo: 0.017940
iteraciones: 50 costo: 0.014070
iteraciones: 60 costo: 0.011577
iteraciones: 70 costo: 0.009835
iteraciones: 80 costo: 0.008550
iteraciones: 90 costo: 0.007563
iteraciones: 100 costo: 0.006780
iteraciones: 110 costo: 0.006327
iteraciones: 120 costo: 0.005948
iteraciones: 130 costo: 0.005613
iteraciones: 140 costo: 0.005313
iteraciones: 150 costo: 0.005044



## Visualización de los vectores palabra
En esta parte visualizarás los vectores de palabras entrenados usando la función que acabas de codificar arriba.

In [54]:
# visualizar las palabras
from matplotlib import pyplot as pyplot

In [55]:
embs = (W1.T + W2)/2

In [57]:
words=['king', 'queen', 'lord', 'man']

In [58]:
idx = [word2Ind[word] for word in words]

In [59]:
X = embs[idx, :]

In [60]:
X

array([[0.54521446, 0.41014684, 0.41737463, 0.29086396, 0.47370344,
        0.38804025, 0.16611713, 0.0675407 , 0.54882368, 0.50095014,
        0.30666447, 0.37014731, 0.81487152, 0.16622548, 0.52666188,
        0.34889787, 0.31079976, 0.41048643, 0.41657351, 0.32937671,
        0.34793808, 0.76784687, 0.62225475, 0.59038878, 0.50140923,
        0.85203053, 0.37792251, 0.46264225, 0.73724502, 0.55377457,
        0.47764351, 0.61771315, 0.26853993, 0.45079032, 0.22959347,
        0.23329301, 0.2039222 , 0.14412412, 0.24056453, 0.42619437,
        0.78883867, 0.86044288, 0.58462057, 0.34632325, 0.77916666,
        0.1079032 , 0.78175782, 0.89290965, 0.28650022, 0.23362268],
       [0.80354897, 0.83606993, 0.34871405, 0.42569186, 0.40437955,
        0.40157391, 0.35057797, 0.68894489, 0.31512585, 0.67068405,
        0.57703794, 0.35542856, 0.55043073, 0.37642296, 0.57522282,
        0.50835822, 0.57720865, 0.91022614, 0.56313168, 0.40183679,
        0.38102435, 0.44578884, 0.53772782, 0.2

In [ ]:
# Como visualizar los vectores
